In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import simsopt


In [ ]:
from simsopt.mhd import Vmec

equil = Vmec("qifiles/wout_QI_nfp3.nc")
equil = Vmec("../tests/test_files/wout_n3are_R7.75B5.7.nc")
surf = equil.boundary
surf.plot()


In [ ]:
import simsopt.mhd

def B_avg_volume(vmec:simsopt.mhd.Vmec, s, nphi=64, ntheta=47):
  vs = simsopt.mhd.vmec_splines(vmec)
  nfp = vs.nfp
  # Shorthand:
  mnmax_nyq = vs.mnmax_nyq
  xm_nyq = vs.xm_nyq
  xn_nyq = vs.xn_nyq

  theta = np.linspace(0, 2*np.pi,   ntheta, False)
  phi = np.linspace(0, 2*np.pi/nfp, nphi, False)

  try:
      ns = len(s)
  except:
      s = [s]
  s = np.array(s)
  ns = len(s)
  # Handle theta
  try:
      ntheta = len(theta)
  except:
      theta = [theta]
  theta_vmec = np.array(theta)
  if theta_vmec.ndim == 1:
      ntheta = len(theta_vmec)
  elif theta_vmec.ndim == 3:
      ntheta = theta_vmec.shape[1]
  else:
      raise ValueError("theta argument must be a float, 1d array, or 3d array.")

  # Handle phi
  try:
      nphi = len(phi)
  except:
      phi = [phi]
  phi = np.array(phi)
  if phi.ndim == 1:
      nphi = len(phi)
  elif phi.ndim == 3:
      nphi = phi.shape[2]
  else:
      raise ValueError("phi argument must be a float, 1d array, or 3d array.")

  # If theta and phi are not already 3D, make them 3D:
  if theta_vmec.ndim == 1:
      theta_vmec = np.kron(np.ones((ns, 1, nphi)), theta_vmec.reshape(1, ntheta, 1))
  if phi.ndim == 1:
      phi = np.kron(np.ones((ns, ntheta, 1)), phi.reshape(1, 1, nphi))

  gmnc = np.zeros((ns, mnmax_nyq))
  bmnc = np.zeros((ns, mnmax_nyq))
  for jmn in range(mnmax_nyq):
    gmnc[:, jmn] = vs.gmnc[jmn](s)
    bmnc[:, jmn] = vs.bmnc[jmn](s)
    

  angle = xm_nyq[:, None, None, None] * theta_vmec[None, :, :, :] - xn_nyq[:, None, None, None] * phi[None, :, :, :]
  cosangle = np.cos(angle)
  modB = np.einsum('ij,jikl->ikl', bmnc, cosangle)
  sqrt_g_vmec = np.einsum('ij,jikl->ikl', gmnc, cosangle)

  # Surface average of the quantity
  dtheta = 2*np.pi / ntheta
  dphi = 2*np.pi / nfp / nphi
  V_prime = nfp * dtheta * dphi * np.sum(sqrt_g_vmec, axis=(1, 2))
  fsa_X = (np.sum(modB * sqrt_g_vmec, axis=(1, 2)) / V_prime * nfp * dtheta * dphi)

  return fsa_X


In [ ]:
plt.plot(B_avg_volume(equil, np.linspace(0,1,10)))
equil.wout.volavgB, np.mean(B_avg_volume(equil, np.linspace(0,1,10)))


In [ ]:
def Bv_over_Bl_analytical(R, aspect_ratio):
  Bv = 2 *aspect_ratio / R * (aspect_ratio - np.sqrt(aspect_ratio**2-1))
  Bl = 1/R
  mu_0 = 1
  I_total = 1
  Bv *= mu_0 * I_total / np.pi
  Bl *= mu_0 * I_total / np.pi
  return Bv/Bl

Bv = equil.wout.volavgB
Bv = np.mean(B_avg_volume(equil, np.linspace(0,1,10)))
Bl = np.mean(B_avg_volume(equil, np.linspace(0,.05,4)))

Bv/Bl, Bv_over_Bl_analytical(equil.wout.Rmajor_p, equil.aspect())


In [ ]:
print(equil.wout.Rmajor_p, equil.wout.Aminor_p, equil.wout.b0, equil.wout.volavgB)
print(equil.wout.Rmajor_p / equil.wout.Aminor_p)
Bv_over_Bl_analytical(equil.wout.Rmajor_p, equil.aspect())


In [ ]:
ref = []
ref2 = []
analytic = []

configurations = [
# "../tests/test_files/wout_10x10.nc",
"../tests/test_files/wout_20220102-01-053-003_QH_nfp4_aspect6p5_beta0p05_iteratedWithSfincs_reference.nc",
"../tests/test_files/wout_ITERModel_reference.nc",
"../tests/test_files/wout_LandremanPaul2021_QA_lowres.nc",
"../tests/test_files/wout_LandremanPaul2021_QA_reactorScale_lowres_reference.nc",
"../tests/test_files/wout_LandremanPaul2021_QH_reactorScale_lowres_reference.nc",
"../tests/test_files/wout_LandremanSengupta2019_section5.4_B2_A80_reference.nc",
# "../tests/test_files/wout_LandremanSenguptaPlunk_section5p3_reference.nc",
"../tests/test_files/wout_W7-X_without_coil_ripple_beta0p05_d23p4_tm_reference.nc",
"../tests/test_files/wout_c09r00_fixedBoundary_0.5T_vacuum_ns201.nc",
"../tests/test_files/wout_circular_tokamak_aspect_100_reference.nc",
"../tests/test_files/wout_circular_tokamak_reference.nc",
"../tests/test_files/wout_li383_low_res_reference.nc",
"../tests/test_files/wout_n3are_R7.75B5.7.nc",
"../tests/test_files/wout_n3are_R7.75B5.7_lowres.nc",
# "../lgradb_normalization/qifiles/wout_nfp3_beta_1.50.nc",
# "../lgradb_normalization/qifiles/wout_nfp3_beta_1.00.nc",
# "../lgradb_normalization/qifiles/wout_nfp3_beta_0.50.nc",
"../lgradb_normalization/qifiles/wout_QI_nfp3.nc",
# "../lgradb_normalization/qifiles/wout_nfp3_beta_3.75.nc",
# "../lgradb_normalization/qifiles/wout_nfp3_beta_3.50.nc",
# "../lgradb_normalization/qifiles/wout_nfp3_beta_3.25.nc",
# "../lgradb_normalization/qifiles/wout_nfp3_beta_3.00.nc",
# "../lgradb_normalization/qifiles/wout_nfp3_beta_2.75.nc",
# "../lgradb_normalization/qifiles/wout_nfp3_beta_2.50.nc",
# "../lgradb_normalization/qifiles/wout_nfp3_beta_2.25.nc",
# "../lgradb_normalization/qifiles/wout_nfp3_beta_2.00.nc",
# "../lgradb_normalization/qifiles/wout_nfp3_beta_1.75.nc",
# "../lgradb_normalization/qifiles/wout_nfp3_beta_1.25.nc",
# "../lgradb_normalization/qifiles/wout_nfp3_beta_0.75.nc",
# "../lgradb_normalization/qifiles/wout_nfp3_beta_0.25.nc"
]
for conf_path in configurations:
  print(conf_path)
  equil = Vmec(conf_path)


  # Flux surface averages for b
  B_fsa = B_avg_volume(equil, np.linspace(0,1,10))
  Bv = np.mean(B_fsa)
  Bl = np.mean(B_fsa[:3])

  ref.append(Bv/Bl)
  ref2.append(equil.wout.volavgB/Bl)
  analytic.append(Bv_over_Bl_analytical(equil.wout.Rmajor_p, equil.aspect()))
  print("VMEC: ", ref[-1],"Analytical approximation: ", analytic[-1])
  # plt.plot(B_fsa, label=conf_path)
  # plt.legend()
  # plt.show()

plt.scatter(ref, analytic)
plt.scatter(ref2, analytic)
plt.xlabel("Numerical solution")
plt.ylabel("Analytic approximation")
